<a href="https://colab.research.google.com/github/AniruddhS24/ArithmeticEvaluator/blob/master/linsvm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SVM stuff

In [ ]:
class SVMMultiClass:
    def __init__(self, gram, tf_idf, pairs, svms, vocab: Indexer, labels: Indexer, train_x, train_y):
        self.gram = gram
        self.tf_idf = tf_idf
        self.pairs = pairs
        self.svms = svms
        self.vocab = vocab
        self.labels = labels
        self.train_x = train_x
        self.train_y = train_y

    def process_input(self, x):
        x_arr = np.zeros(len(self.vocab))
        for i in range(0, len(x) - self.gram + 1):
            if (self.vocab.contains(x[i:i + self.gram])):
                x_arr[self.vocab.index_of(x[i:i + self.gram])] += 1
        if self.tf_idf:
            x_arr /= np.sum(x_arr)
            for w in range(len(x_arr)):
                if np.sum((self.train_x[:, w] > 0)) > 0:
                    x_arr[w] *= np.log(self.train_x.shape[1] / np.sum((self.train_x[:, w] > 0)))
        return x_arr

    def predict(self, x, targets):
        x = self.process_input(x)
        votes = np.zeros(len(self.labels))
        for i in range(len(self.svms)):
            pd = self.svms[i].predict(x)
            votes[self.pairs[i][pd]]+=1

        maxvotes = 0
        res = targets[0]
        for target in targets:
            if votes[self.labels.index_of(target)] > maxvotes:
                maxvotes = votes[self.labels.index_of(target)]
                res = target

        return res